In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import torchaudio

from model.SpeechGenerator import SpeechGenerator
from utils.Config import ConfigSLP, ConfigNAC, ConfigDiTTO
from utils.MLS import MLSDataset
from utils.Processing import Processing

from torch.utils.data import DataLoader

In [46]:
ConfigSLP.display()
ConfigNAC.display()
ConfigDiTTO.display()

############# Base Settings: #############

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Training Settings:
  Betas: [0.9, 0.999]
  Device: cuda

Data Settings:
  Train Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/train
  Test Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/test
  Dev Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/dev

############# SLP Settings: #############

Model Settings:
  Model Name: SLP
  Embedding Dimension: 1472
  Number of Layers: 1
  Number of Attention Heads: 1
  Number of Classes: 11
  Number of Samples: 10000
  Batch Size: 8
  Learning Rate: 0.0001
  Epochs: 20
  Token length for ByT5: 128
############# Base Settings: #############

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Training Settings:
  Betas: [0.9, 0.999]
  Device: cuda

Data Settings:
  Train Path: /tempory/M2-DA

In [ ]:
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TRAIN_PATH+"/"+"audio", ConfigSLP.TRAIN_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TEST_PATH+"/"+"audio", ConfigSLP.TEST_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.DEV_PATH+"/"+"audio", ConfigSLP.DEV_PATH+"/"+"audio_clean",)

## Speech Generation with DiTTO-TTs and Vocoder

In [ ]:
train_set = MLSDataset(
    data_dir=ConfigDiTTO.TRAIN_PATH,
    max_text_token_length=ConfigDiTTO.MAX_TOKEN_LENGTH,
    sampling_rate=ConfigDiTTO.SAMPLE_RATE,
    nb_samples=ConfigDiTTO.NB_SAMPLES,
    tokenizer_model="gpt2"
)

test_set = MLSDataset(
    data_dir=ConfigDiTTO.TEST_PATH,
    max_text_token_length=ConfigDiTTO.MAX_TOKEN_LENGTH,
    sampling_rate=ConfigDiTTO.SAMPLE_RATE,
    nb_samples=ConfigDiTTO.NB_SAMPLES,
    tokenizer_model="gpt2"
)

train_loader = DataLoader(train_set, batch_size=ConfigNAC.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)
test_loader = DataLoader(test_set, batch_size=ConfigNAC.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)

In [ ]:
speech_generator = SpeechGenerator(
    nac_model_path="/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/params/NAC_epoch_20.pth",
    ditto_model_path="/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/params/DiTTO_epoch_20.pth",
    lambda_factor=ConfigNAC.LAMBDA_FACTOR,
    sample_rate=ConfigNAC.SAMPLE_RATE,
    device=ConfigDiTTO.DEVICE
)

In [47]:
def test_with_loader(loader, prompt):
    device = ConfigDiTTO.DEVICE
    item = next(iter(loader))
    item["text"]["input_ids"] = item["text"]["input_ids"].to(device)
    item["text"]["attention_mask"] = item["text"]["attention_mask"].to(device)
    text_input = item["text"]["input_ids"][0].unsqueeze(0)
    audio_input = item["audio"][0].to(device).unsqueeze(0)
    padding_mask_audio = item["padding_mask_audio"][0].to(device).unsqueeze(0)
    generated_waveform = speech_generator.generate_speech_from_audio_tensor(audio_input, padding_mask_audio, prompt)
    output_path = "output.wav"
    torchaudio.save(output_path, generated_waveform.cpu(), ConfigDiTTO.SAMPLE_RATE)

In [48]:
test_with_loader(train_loader, "Bonjour, comment çava tout le monde ?")

1000it [04:49,  3.45it/s]


[WARNING] Min value of input waveform signal is -1.2479965686798096


In [ ]:
test_with_loader(test_loader, "Bonjour, comment çava tout le monde ?")